In [ ]:
import openai
import instructor
from qdrant_client import QdrantClient
from pydantic import BaseModel, Field

In [ ]:
# Basic example that shows how to use instructor 
prompt = """How is the weather today in Italy?"""

# create the instructor client for the openai client
client = instructor.from_openai(openai.OpenAI())

In [ ]:
# Define the output schema using Pydantic. This schema will be used to structure the model's response via instructor.
class StructuredResponse(BaseModel):
    answer: str = Field(..., description="A brief summary of the weather in Italy today.")

In [ ]:
# Now invoke the model with the prompt and specify the response schema.
# you can use either the "create" method or the "create_with_completion" method; the difference is that the latter also returns the original LLM response.
# This is useful for debugging but also to retrieve additional information such as token usage, and then wraps them into the model response.
response, original_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
    temperature=0,
    response_model=StructuredResponse,
)

In [ ]:
response

In [ ]:
original_response

In [ ]:
# Now let's define a sample RAG pipeline
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(
        context["retrieved_context_ids"],
        context["retrieved_context"],
        context["retrieved_context_ratings"],
    ):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

# Instead of using the Openai client, let's use the instructor client here as well
# and the StructuredResponse model defined above
def generate_answer(prompt):

    response, original_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=StructuredResponse,
    )
    # the return object is the StructuredResponse model instance
    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    response = generate_answer(prompt)

    final_result = {
        # print the full data model response for reference (debugging/tracing)
        "data_model": response,
        "answer": response.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "retrieved_context_ratings": retrieved_context["retrieved_context_ratings"],
        "similarity_scores": retrieved_context["similarity_scores"],
    }

    return final_result

In [ ]:
# create the Qdrant client (ensure to start the container first)
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
# let's invoke the rag pipeline
query = "What are some good products with high ratings for outdoor activities?"
output = rag_pipeline(query, qdrant_client)

In [ ]:
output